In [1]:
# Entropy
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    """Given a list of class probabilities, compute the entropy"""
    
    return sum(-p * math.log(p, 2) 
               for p in class_probabilities 
               if p > 0) # ignore zero probabilities

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

In [2]:
from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    
    return [count / total_count 
            for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])

In [3]:
# Entropy of partition (weighted sum of partition entropy)

def partition_entropy(subsets: List[List[Any]]) -> float:
    """Returns the entropy from this partition of data into subsets"""
    
    total_count = sum(len(subset) for subset in subsets)
    
    return sum(data_entropy(subset) * (len(subset) / total_count) 
               for subset in subsets) 

In [4]:
# Data

from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None # allow unlabeled data

                 #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

In [5]:
# Creating a Decision Tree

from typing import Dict, TypeVar
from collections import defaultdict

T = TypeVar('T') # generic type for inputs

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """Partition the inputs into lists based on the specified attribute"""
    
    partitions: Dict[Any, List[T]] = defaultdict(list)
    
    for input in inputs:
        key = getattr(input, attribute) # value of the specified attribute
        partitions[key].append(input)   # add input to the correct partition
    
    return partitions

def partition_entropy_by(inputs: List[Any], attribute: str, label_attribute: str) -> float:
    """Compute the entropy corresponding to the given partition"""
    
    partitions = partition_by(inputs, attribute) # partitions consist of our inputs
    labels = [[getattr(input, label_attribute) for input in partition] # but partition_entropy needs just the class labels 
              for partition in partitions.values()]
    
    return partition_entropy(labels)

In [6]:
# Finding the minimum entropy partition

for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key, 'did_well')) # the lowest entropy comes from splitting on 'level'

level 0.6935361388961918
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [7]:
# Building a subtree for each possible level value

senior_inputs = [input for input in inputs 
                 if input.level == 'Senior']

for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(senior_inputs, key, 'did_well')) # the lowest entropy comes from splitting on 'tweets'

lang 0.4
tweets 0.0
phd 0.9509775004326937


In [8]:
# Definition of Decision Tree

from typing import NamedTuple, Union, Any

class Leaf(NamedTuple):
    value: Any
        
class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None # handle an unknown value
        
DecisionTree = Union[Leaf, Split]

hiring_tree = Split('level', {   # first, consider 'level'
    'Junior': Split('phd', {     # if level is "Junior", next look at 'phd' 
        False: Leaf(True),       # if phd is "False", predict True
        True: Leaf(False)        # if phd is "True", predict False
    }),
    
    'Mid': Leaf(True),           # if level is "Mid", just predict True 
    
    'Senior': Split('tweets', {  # if level is "Senior", next look at "tweets"
        False: Leaf(False),      # if tweets is "False", predict False
        True: Leaf(True)         # if tweets is "True", predict True
    })
})

In [9]:
def classify(tree: DecisionTree, input: Any) -> Any:
    """Classify the input using the given decision tree"""
    
    # If this is a leaf node, return its value
    if isinstance(tree, Leaf):
        return tree.value
    
    # Otherwise this tree consists of an attribute to split on
    # and a dictionary whose keys are values of that attribute
    # and whose values are subtrees to consider next
    
    subtree_key = getattr(input, tree.attribute)
    
    if subtree_key not in tree.subtrees: # If no subtree for key (e.g., "Intern" level)
        return tree.default_value        # return the default value 
    
    subtree = tree.subtrees[subtree_key] # Choose an appropriate subtree
                                         # and use it to classify the input
    
    return classify(subtree, input)

In [10]:
# Creating a Decision Tree from data

def build_tree_id3(inputs: List[Any], 
                   split_attributes: List[str], 
                   target_attribute: str) -> DecisionTree:
    
    # Count target labels
    labels_count = Counter(getattr(input, target_attribute) 
                           for input in inputs)
    most_common_label = labels_count.most_common(1)[0][0]
    
    # If there is a unique label, predict it (no need to learn)
    if len(labels_count) == 1:
        return Leaf(most_common_label)
    
    # If no split attributes left, return the majority label
    if not split_attributes:
        return Leaf(most_common_label)
    
    # Otherwise split by the best attribute
    
    def split_entropy(attribute: str) -> float:
        """Helper function for finding the best attribute to split"""
        
        return partition_entropy_by(inputs, attribute, target_attribute)
    
    best_attribute = min(split_attributes, key=split_entropy)
    
    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes 
                      if a != best_attribute] # eliminate the best attribute we already used
    
    # Recursively build the subtrees
    subtrees = {attribute_value : build_tree_id3(subset, 
                                                 new_attributes, 
                                                 target_attribute)
               for attribute_value, subset in partitions.items()}
    
    return Split(best_attribute, subtrees, default_value=most_common_label)

In [11]:
tree = build_tree_id3(inputs, 
                      ['level', 'lang', 'tweets', 'phd'], 
                      'did_well')

assert classify(tree, Candidate("Junior", "Java", True, False))    # True
assert not classify(tree, Candidate("Junior", "Java", True, True)) # False
assert classify(tree, Candidate("Intern", "Java", True, False))    # True (unexpected value "Intern")

In [12]:
tree

Split(attribute='level', subtrees={'Senior': Split(attribute='tweets', subtrees={False: Leaf(value=False), True: Leaf(value=True)}, default_value=False), 'Mid': Leaf(value=True), 'Junior': Split(attribute='phd', subtrees={False: Leaf(value=True), True: Leaf(value=False)}, default_value=True)}, default_value=True)